In [1]:
import torch as tp 
import numpy as np 
import pandas as pd 
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.model_selection import train_test_split as tts 
from sklearn.datasets import load_breast_cancer


In [2]:
df=pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [3]:
df.drop(columns=['id','Unnamed: 32'],inplace=True)


In [4]:
df

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,...,25.380,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,...,24.990,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902
2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,...,23.570,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758
3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,...,14.910,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300
4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,...,22.540,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,M,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,0.1726,...,25.450,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115
565,M,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,0.1752,...,23.690,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637
566,M,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,0.1590,...,18.980,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820
567,M,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,0.2397,...,25.740,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400


In [5]:
X=df.iloc[:,1:]
y=df.iloc[:,0]

In [6]:
X.head()

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [7]:
y.head()

0    M
1    M
2    M
3    M
4    M
Name: diagnosis, dtype: object

## Train TEst Split

In [8]:
X_train,X_test,y_train,y_test=tts(X,y,test_size=0.2,random_state=42)

## Standardizing data

In [9]:
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)

In [10]:
le=LabelEncoder()
y_train=le.fit_transform(y_train)
y_test=le.transform(y_test)

In [11]:
X_train_tensor=tp.from_numpy(X_train)
X_test_tensor=tp.from_numpy(X_test)
y_train_tensor=tp.from_numpy(y_train)
y_test_tensor=tp.from_numpy(y_test)

In [12]:
X_train_tensor.shape

torch.Size([455, 30])

## Model Training

In [13]:
class SimpleNN:
    def __init__(self,X):

        self.weights=tp.rand(X.shape[1],1,dtype=tp.float64,requires_grad=True)
        self.bias=tp.zeros(1,dtype=tp.float64,requires_grad=True)
    
    def forward(self,X):
        z=tp.matmul(X,self.weights)+self.bias
        y_pred=tp.sigmoid(z)
        return y_pred
    
    def loss_func(self,y_pred,y):
        ## Clamp prediction to avoid log(0)
        epsilon=1e-7
        y_pred=tp.clamp(y_pred,epsilon,1-epsilon)

        ## Calculate loss
        loss=-(y_train_tensor*tp.log(y_pred)+(1-y_train_tensor)*tp.log(1-y_pred)).mean()
        return loss

## Parameters

In [14]:
l_rate=0.1
epochs=100

## Training Pipeline

In [15]:
model=SimpleNN(X_train_tensor)

# for loop 
for epoch in range(epochs):
    
# forward pass
    y_pred=model.forward(X_train_tensor)
    
# loss calc
    loss=model.loss_func(y_pred,y_train_tensor)
    
# backward pass
    loss.backward()
# parameter Update
    with tp.no_grad():
        model.weights-=l_rate*model.weights.grad
        model.bias-=l_rate*model.bias.grad

# Zero GRad
    model.weights.grad.zero_()
    model.bias.grad.zero_()

# Epochs
    print(f'Epochs: {epoch+1}, loss: {loss.item()}')

Epochs: 1, loss: 3.384878772533639
Epochs: 2, loss: 3.2483709211979397
Epochs: 3, loss: 3.1083050775820333
Epochs: 4, loss: 2.962547573126434
Epochs: 5, loss: 2.8121532575053583
Epochs: 6, loss: 2.653271799819786
Epochs: 7, loss: 2.485637650569734
Epochs: 8, loss: 2.3151746610898916
Epochs: 9, loss: 2.149361650517709
Epochs: 10, loss: 1.9863194986200505
Epochs: 11, loss: 1.822047346728697
Epochs: 12, loss: 1.6659882538994069
Epochs: 13, loss: 1.516215162723817
Epochs: 14, loss: 1.3767866411215555
Epochs: 15, loss: 1.249384452296609
Epochs: 16, loss: 1.1386074019969166
Epochs: 17, loss: 1.0455564373717356
Epochs: 18, loss: 0.9704466602827673
Epochs: 19, loss: 0.9123615147471341
Epochs: 20, loss: 0.8692044905978791
Epochs: 21, loss: 0.8380382386629953
Epochs: 22, loss: 0.8157160523858943
Epochs: 23, loss: 0.799471694830162
Epochs: 24, loss: 0.787215506135629
Epochs: 25, loss: 0.7775312801891394
Epochs: 26, loss: 0.769523029818991
Epochs: 27, loss: 0.7626447244969532
Epochs: 28, loss: 0.7

In [16]:
model.bias

tensor([-0.4747], dtype=torch.float64, requires_grad=True)

In [17]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# -------- Step 1: Sample Data --------
X = torch.tensor([[0.0, 0.0],
                  [0.0, 1.0],
                  [1.0, 0.0],
                  [1.0, 1.0],
                  [0.5, 0.5],
                  [0.2, 0.8],
                  [0.8, 0.2],
                  [0.9, 0.9],
                  [0.1, 0.3],
                  [0.3, 0.1]], dtype=torch.float32)

y = torch.tensor([0, 1, 1, 0, 0, 1, 1, 0, 0, 1], dtype=torch.float32).view(-1, 1)

dataset = TensorDataset(X, y)
loader = DataLoader(dataset, batch_size=2, shuffle=True)

# -------- Step 2: Define Model using Sequential --------
model = nn.Sequential(
    nn.Linear(2, 4),   # Input layer -> Hidden layer # 4 Neurons
    nn.ReLU(),          # Activation
    nn.Linear(4, 1),   # Hidden layer -> Output layer # 1 neurons
    nn.Sigmoid()        # Output activation for binary classification
)

# -------- Step 3: Loss and Optimizer --------
criterion = nn.BCELoss()               # Binary Cross Entropy
optimizer = optim.Adam(model.parameters(), lr=0.01)

# -------- Step 4: Training Loop --------
epochs = 100
for epoch in range(epochs):
    for batch_X, batch_y in loader:
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
    
    if (epoch+1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

# -------- Step 5: Testing --------
with torch.no_grad():
    predictions = model(X)
    predicted_classes = (predictions > 0.5).float()
    print("Predicted classes:", predicted_classes.view(-1))
    print("Actual classes   :", y.view(-1))


Epoch [10/100], Loss: 0.6904
Epoch [20/100], Loss: 0.6942
Epoch [30/100], Loss: 0.3921
Epoch [40/100], Loss: 0.7128
Epoch [50/100], Loss: 0.2139
Epoch [60/100], Loss: 0.5911
Epoch [70/100], Loss: 0.4067
Epoch [80/100], Loss: 0.4468
Epoch [90/100], Loss: 0.4297
Epoch [100/100], Loss: 0.2006
Predicted classes: tensor([0., 1., 1., 0., 0., 1., 1., 0., 0., 1.])
Actual classes   : tensor([0., 1., 1., 0., 0., 1., 1., 0., 0., 1.])
